In [273]:
import sys

# Check if the path already exists in sys.path before adding it
if '/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages' not in sys.path:
    #add path to system
  sys.path.append('/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages')

print(sys.path)

['/Users/jerrybenoit/Desktop/Moveout_Project', '/Library/Frameworks/Python.framework/Versions/3.12/lib/python312.zip', '/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12', '/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/lib-dynload', '', '/Users/jerrybenoit/Library/Python/3.12/lib/python/site-packages', '/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages']


In [274]:

from splinter import Browser
from selenium import webdriver
from bs4 import BeautifulSoup as soup
import re
import pandas as pd
import matplotlib.pyplot as plt
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
from datetime import datetime
from pymongo import MongoClient 

# Get the current date
current_date = datetime.now().strftime("%Y-%m-%d-%H-%M") 
print(current_date)

collection_name = f"appartments_{current_date}"

client = MongoClient("mongodb://localhost:27017/")

#assigne une bd qui existe si ca n'existe pas il va etre crée
db = client["Appartements_moveout"]

collection = db[collection_name]
print(db.list_collection_names())

2024-04-28-14-54
[]


In [275]:
browser = Browser('chrome')
#browser.driver.set_page_load_timeout(30)  # sets the timeout to 30 seconds

# Définition de l'URL de base
base_url = "https://www.facebook.com/marketplace/montreal/search?"

# Définition des paramètres de recherche pour les appartements
min_price = 300  # Prix minimum en dollars
max_price = 1000  # Prix maximum en dollars
min_bedrooms = 2  # Nombre minimum de chambres
min_bathrooms = 1  # Nombre minimum de salles de bains
min_sqft = 500  # Superficie minimum en pieds carrés
max_sqft = 2000  # Superficie maximum en pieds carrés
days_listed = 7  # Nombre de jours depuis la mise en ligne
pets_allowed = True  # Animaux autorisés
furnished = False  # Meublé ou non
neighborhood = "Montreal"  # Quartier souhaité
#attempt to only show appartment on the feed
recherche = "appartment"




In [276]:
for bedrooms in range(1, 3):

    print(f"bedrooms: {bedrooms}")

    # Construction de l'URL complet en formatant correctement la chaîne de requête
    #url = f"{base_url}minPrice={min_price}&maxPrice={max_price}&minBedrooms={min_bedrooms}&minBathrooms={min_bathrooms}&minSqft={min_sqft}&maxSqft={max_sqft}&daysSinceListed={days_listed}&query={min_bedrooms}+bedroom+{recherche}&exact=false"


    urls =[
    #
    f"{base_url}minPrice={min_price}&maxPrice={1000}&minBedrooms={bedrooms}&minBathrooms={min_bathrooms}&minSqft={min_sqft}&maxSqft={max_sqft}&daysSinceListed={days_listed}&query={min_bedrooms}+bedroom+{recherche}&exact=false",
    f"{base_url}minPrice={min_price}&maxPrice={1500}&minBedrooms={bedrooms}&minBathrooms={min_bathrooms}&minSqft={min_sqft}&maxSqft={max_sqft}&daysSinceListed={days_listed}&query={min_bedrooms}+bedroom+{recherche}&exact=false",
    f"{base_url}minPrice={1500}&maxPrice={3000}&minBedrooms={bedrooms}&minBathrooms={min_bathrooms}&minSqft={min_sqft}&maxSqft={max_sqft}&daysSinceListed={days_listed}&query={min_bedrooms}+bedroom+{recherche}&exact=false"
    ]

    for url in urls:
        print(f"visiting: URL: {url}")  

        # Visit the website
        browser.visit(url)
        print("Url Visité:")

        # Scroll down to load more results

        #define the number of times to scroll the page
        scroll_count = 4

        #Define the delay (in seconds) between each scroll
        scroll_delay = 2

        #Loop to perform scrolling
        for _ in range(scroll_count):
          #Execute Javascript to scroll to the bottom of the page 
          browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

          time.sleep(scroll_delay)

        # Parse the HTML
        html = browser.html
        # Create a BeautifulSoup object from the scraped HTML
        market_soup = soup(html, 'html.parser')
        # Check if HTML was scraped correctly

        #market_soup

       
        #merde  qui sont pas des appartements donc je les mets dans une liste pour supprimer plus tard
        elements_a_supprimer = ['Filters', 'Categories']


        #Extract all the necessary info and insert into lists
        titles_div = market_soup.find_all('span', class_="x1lliihq x6ikm8r x10wlt62 x1n2onr6")

        #for loop that goes everything inside titles div & stripping everything that not pure text
        titles_list = [title.text.strip() for title in titles_div]

        #Nouvelle liste cleans
        titles_clean = [title for title in titles_list if title not in elements_a_supprimer]

        prices_div = market_soup.find_all('span', class_="x193iq5w xeuugli x13faqbe x1vvkbs xlh3980 xvmahel x1n0sxbx x1lliihq x1s928wv xhkezso x1gmr53x x1cpjm7i x1fgarty x1943h6x x4zkp8e x3x7a5m x1lkfr7t x1lbecb7 x1s688f xzsf02u")
        prices_list = [price.text.strip() for price in prices_div]

        city_div = market_soup.find_all('span', class_="x193iq5w xeuugli x13faqbe x1vvkbs xlh3980 xvmahel x1n0sxbx x1lliihq x1s928wv xhkezso x1gmr53x x1cpjm7i x1fgarty x1943h6x x4zkp8e x676frb x1nxh6w3 x1sibtaa xo1l8bm xi81zsa")
        city_list = [city.text.strip() for city in city_div]

        urls_div = market_soup.find_all('a',class_="x1i10hfl xjbqb8w x1ejq31n xd10rxx x1sy0etr x17r0tee x972fbf xcfux6l x1qhh985 xm0m39n x9f619 x1ypdohk xt0psk2 xe8uvvx xdj266r x11i5rnm xat24cr x1mh8g0r xexx8yu x4uap5 x18d9i69 xkhd6sd x16tdsg8 x1hl2dhg xggy1nq x1a2a7pz x1heor9g x1lku1pv")
        urls_list = [url.get('href') for url in urls_div]

        img_div = market_soup.find_all('img', class_="xt7dq6l xl1xv1r x6ikm8r x10wlt62 xh8yej3")
        img_list = [img.get('src') for img in img_div]

        #img_list


        appartment_list = []

        min_length = min(len(titles_list), len(prices_list), len(city_list), len(urls_list))

        for title, price, city, url, img in zip(titles_clean, prices_list, city_list, urls_list,img_list):
          room_dict = {}
          
        
          apartment_data = {
            
                        "title": title.strip(),
                  
                        "price": int(re.sub(r'[^\d.]', '', price.strip())),
                        "city": city.strip(),
                        "bedrooms": bedrooms,
                        "url": 'https://www.facebook.com/' + url,
                        "img": img,
                        "date_scraped": datetime.now()
                        
                    }
          print("p: ", apartment_data)
                      
      
# End the automated browsing session
browser.quit()

    



bedrooms: 1
visiting: URL: https://www.facebook.com/marketplace/montreal/search?minPrice=300&maxPrice=1000&minBedrooms=1&minBathrooms=1&minSqft=500&maxSqft=2000&daysSinceListed=7&query=2+bedroom+appartment&exact=false
Url Visité:
p:  {'title': '2 chambres à coucher 1 salle de bain Appartement', 'price': 835, 'city': 'Montréal, QC', 'bedrooms': 1, 'url': 'https://www.facebook.com//marketplace/item/792252272846241/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD', 'img': 'https://scontent-yyz1-1.xx.fbcdn.net/v/t45.5328-4/438062752_2055886178129239_3930723835427667755_n.jpg?stp=c80.0.480.480a_dst-jpg&_nc_cat=104&ccb=1-7&_nc_sid=247b10&_nc_ohc=YqpFufDZzDoAb4iSvo5&_nc_ht=scontent-yyz1-1.xx&oh=00_AfDXcz-ZkjN-ArRM09C3COmGXfdgafcUX6ArsKNR2QN_6w&oe=6634728A', 'date_scraped': datetime.datetime(2024, 4, 28, 14, 54, 32, 676345)}
p:  {'title': '2 Beds 2 Baths - Apartment', 'price': 890, 'city': 'Montréal, QC', 'bedrooms': 1, 'url': 'https://www.facebook.com//marketplace/item/150

In [277]:
appartment_list

[]

In [278]:
len(appartment_list)

0

In [279]:
appartment_list_df = pd.DataFrame(appartment_list)

In [280]:
appartment_list_df

""


In [281]:
appartment_list_df['URL'] = 'https://www.facebook.com/' + appartment_list_df['URL']

KeyError: 'URL'

In [ ]:
appartment_list_df

,title,Price,City,URL,Image
0,2 Beds 1 Bath - Apartment,750,"Montréal, QC",https://www.facebook.com//marketplace/item/641...,https://scontent-yyz1-1.xx.fbcdn.net/v/t39.308...
1,2 Beds 1 Bath Apartment,700,"Montréal, QC",https://www.facebook.com//marketplace/item/927...,https://scontent-yyz1-1.xx.fbcdn.net/v/t45.532...
2,2 Beds 2 Baths - Apartment,890,"Montréal, QC",https://www.facebook.com//marketplace/item/150...,https://scontent-yyz1-1.xx.fbcdn.net/v/t45.532...
3,2 Beds 1 Bath Apartment,875,"Montréal, QC",https://www.facebook.com//marketplace/item/111...,https://scontent-yyz1-1.xx.fbcdn.net/v/t45.532...
4,2 Beds 1 Bath Apartment,750,"Montréal, QC",https://www.facebook.com//marketplace/item/389...,https://scontent-yyz1-1.xx.fbcdn.net/v/t39.308...
...,...,...,...,...,...
65,Appartement de 1 chambre et 1 salle de bain,750,"Montréal, QC",https://www.facebook.com//marketplace/item/384...,https://scontent-yyz1-1.xx.fbcdn.net/v/t45.532...
66,1 Bed 1 Bath - Apartment,840,"L'Île-Perrot, QC",https://www.facebook.com//marketplace/item/123...,https://scontent-yyz1-1.xx.fbcdn.net/v/t39.308...
67,1 Bed 1 Bath - Apartment,940,"Montréal, QC",https://www.facebook.com//marketplace/item/740...,https://scontent-yyz1-1.xx.fbcdn.net/v/t45.532...
68,Appartement de 3 chambres et 1 salle de bain,850,"Montréal, QC",https://www.facebook.com//marketplace/item/816...,https://scontent-yyz1-1.xx.fbcdn.net/v/t45.532...
